### Imports

In [1]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import BerrarManual, BerrarNumerical, BerrarAnalytical
from nera.reference import *
from nera.trainer import Trainer
from nera.utils import print_rating_diff


/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


### Data acquisition

In [2]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
data_transform = DataTransformation(df, timedelta(365))
df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

trainer = Trainer(dataset, train_ratio=1, loss_fn=torch.nn.MSELoss)
reference_maker = RatingReference(transform.num_teams)

berrar_params = {
            'alpha_h': torch.tensor(180, dtype=torch.float64),
            'alpha_a': torch.tensor(180, dtype=torch.float64),
            'beta_h': torch.tensor(2, dtype=torch.float64),
            'beta_a': torch.tensor(2, dtype=torch.float64),
            'bias_h': torch.tensor(0, dtype=torch.float64),
            'bias_a': torch.tensor(0, dtype=torch.float64),
            'lr_h_att': torch.tensor(0.1, dtype=torch.float64),
            'lr_a_att': torch.tensor(0.1, dtype=torch.float64),
            'lr_h_def': torch.tensor(0.1, dtype=torch.float64),
            'lr_a_def': torch.tensor(0.1, dtype=torch.float64),
            'default': 1000.
        }

2024-03-14 14:12:59.099 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


### Manual berrar NN vs reference

In [10]:

computed = reference_maker.compute_reference('berrar', dataset, **berrar_params)

print()
berrar_man = BerrarManual(team_count=transform.num_teams, **berrar_params)
trainer.dataset = dataset
trainer.model = berrar_man
acc_late = trainer.train(epochs=1, val_ratio=0, verbose=True)

err = False
for rating in range(len(computed)):
    for i in range(len(computed[rating])):
        cmp = float(computed[rating][i])
        net = float(berrar_man.ratings[rating][i])
        if abs(cmp - net) > 0.1:
            print(rf'rating {rating}::ERROR on index {i}:: {cmp} / {net}')
            err = True
        if i < 5:
            print(f'rating {rating}::{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed berrar rating is the same as berrar from NN")

2024-03-14 12:25:40.797 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 64783.367, training accuracy: 26.50%


rating 0::0:: computed:    896.000 || net:    896.000
rating 0::1:: computed:     20.717 || net:     20.717
rating 0::2:: computed:     -5.649 || net:     -5.649
rating 0::3:: computed:     12.814 || net:     12.814
rating 0::4:: computed:      2.144 || net:      2.144
rating 1::0:: computed:    891.900 || net:    891.900
rating 1::1:: computed:     16.318 || net:     16.318
rating 1::2:: computed:      3.822 || net:      3.822
rating 1::3:: computed:      4.468 || net:      4.468
rating 1::4:: computed:    -15.918 || net:    -15.918
...
[SUCCESS]: Computed berrar rating is the same as berrar from NN


### Analytical vs numerical pass
1) gradient check

In [4]:
from torch.autograd import gradcheck
from nera.models.ratings._berrar._analytical import _berrar_fn
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(_berrar_fn, input, eps=1e-6, atol=1e-4)
print(test)

True


2. analytical vs numerical nn pass

In [3]:
berrar_numerical = BerrarNumerical(team_count, **berrar_params)
berrar_analytical = BerrarAnalytical(team_count, **berrar_params)

trainer.train_ratio = 0.8 

In [5]:
trainer.model = berrar_numerical
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 